# tootltips

In [8]:
!curl http://localhost:9200/dpe_pre_1948_td005/_count

{"count":0,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0}}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    71  100    71    0     0   1543      0 --:--:-- --:--:-- --:--:--  1543


# Imports

In [2]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk,parallel_bulk

# load table

In [3]:
from generate_dpe_annexes_scripts.utils import reload_package

In [61]:
td001.columns

Index(['id', 'numero_dpe', 'usr_diagnostiqueur_id', 'usr_logiciel_id',
       'tr001_modele_dpe_id', 'nom_methode_dpe', 'version_methode_dpe',
       'nom_methode_etude_thermique', 'version_methode_etude_thermique',
       'date_visite_diagnostiqueur', 'date_etablissement_dpe',
       'date_arrete_tarifs_energies',
       'commentaires_ameliorations_recommandations',
       'explication_personnalisee', 'consommation_energie',
       'classe_consommation_energie', 'estimation_ges',
       'classe_estimation_ges', 'tr002_type_batiment_id', 'secteur_activite',
       'tr012_categorie_erp_id', 'tr013_type_erp_id', 'annee_construction',
       'surface_habitable', 'surface_thermique_lot', 'tv016_departement_id',
       'commune', 'arrondissement', 'type_voie', 'nom_rue', 'numero_rue',
       'batiment', 'escalier', 'etage', 'porte', 'code_postal',
       'code_insee_commune', 'code_insee_commune_actualise', 'numero_lot',
       'quote_part', 'nom_centre_commercial',
       'surface_commerci

In [30]:
import numpy as np

import pandas as pd
from pathlib import Path
import json
from generate_dpe_annexes_scripts.utils import round_float_cols, unique_ordered
from config import paths

if __name__ == '__main__':

    data_dir = paths['DPE_DEPT_PATH']
    annexe_dir = paths['DPE_DEPT_ANNEXE_PATH']
    annexe_dir = Path(annexe_dir)
    annexe_dir.mkdir(exist_ok=True, parents=True)
    for dept_dir in Path(data_dir).iterdir():
        if dept_dir.name=='1':
            print(dept_dir)
            annexe_dept_dir = annexe_dir / dept_dir.name
            annexe_dept_dir.mkdir(exist_ok=True, parents=True)
            # LOAD TABLES
            td001 = pd.read_csv(dept_dir / 'td001_dpe.csv', dtype=str)
            td003 = pd.read_csv(dept_dir / 'td003_descriptif.csv', dtype=str)
            td002 = pd.read_csv(dept_dir/'td002_consommation.csv',dtype=str)
            td016 = pd.read_csv(dept_dir/'td016_facture.csv',dtype=str)

            td005 = pd.read_csv(dept_dir / 'td005_fiche_technique.csv', dtype=str)
            td006 = pd.read_csv(dept_dir / 'td006_batiment.csv', dtype=str)
            td001_enveloppe_agg=pd.read_csv(annexe_dept_dir / 'td001_enveloppe_agg_annexe.csv',dtype=str)
            td007_p = pd.read_csv(annexe_dept_dir / 'td007_paroi_opaque_annexe.csv',dtype=str)
            td008_p = pd.read_csv(annexe_dept_dir / 'td008_baie_annexe.csv')
            td001_td008 = pd.read_csv(annexe_dept_dir / 'td001_td008_agg_annexe.csv')
            td001_td006 = pd.read_csv(annexe_dept_dir / 'td001_td006_agg_annexe.csv', dtype=str)

            gen= pd.read_csv(annexe_dept_dir / 'td001_annexe_generale.csv', dtype=str)
            grnv = pd.read_csv(annexe_dept_dir / 'td001_agg_synthese_gorenove.csv', dtype=str)
            env = pd.read_csv(annexe_dept_dir / 'td001_env_agg_adv.csv', dtype=str)


            print(dept_dir)
            break
            # ENVELOPPE PROCESSING


D:\data\dpe_full\depts\1
D:\data\dpe_full\depts\1


In [33]:
env.mat_mur_ext.fillna('na').value_counts()

murs en blocs de béton creux                                          25863
murs en béton banché                                                  22072
na                                                                    18852
murs en pierre de taille et moellons avec remplissage tout venant      9279
murs en béton                                                          5656
murs en blocs de béton pleins                                          2998
murs en pierre                                                         2990
murs en briques creuses                                                1582
cloison de plâtre                                                      1457
murs en pierre de taille et moellons constitué d'un seul matériaux     1289
murs en béton de mâchefer                                              1055
murs en briques pleines simples                                        1034
murs en pan de bois sans remplissage tout venant                        855
béton cellul

In [43]:
td001 = pd.read_csv(dept_dir / 'td001_dpe.csv', dtype=str)
td001 = td001.rename(columns={'id': 'td001_dpe_id'})
gen_adv = pd.read_csv(annexe_dept_dir / 'td001_gen_agg_adv.csv', dtype=str)
sys_adv = pd.read_csv(annexe_dept_dir / 'td001_sys_agg_adv.csv', dtype=str)
env_surf = pd.read_csv(annexe_dept_dir / 'td001_enveloppe_surface_agg_annexe.csv', dtype=str)
env_surf = env_surf.rename(columns={"Unnamed: 0": "td001_dpe_id"})
env_adv = pd.read_csv(annexe_dept_dir / 'td001_env_agg_adv.csv', dtype=str)
td001_cols = ['td001_dpe_id', 'numero_dpe', 'tr002_type_batiment_id', 'consommation_energie', 'estimation_ges',
              'classe_consommation_energie', 'classe_estimation_ges']

env_surf_cols = ['td001_dpe_id', 'perc_surf_vitree_ext']

sys_cols = ['td001_dpe_id', 'type_installation_ch', 'type_energie_ch', 'gen_ch_lib_final',
            'gen_ch_lib_principal', 'gen_ch_lib_appoint', 'is_ch_solaire', 'type_installation_ecs',
            'type_energie_ecs', 'gen_ecs_lib_final',
            'gen_ecs_lib_principal', 'gen_ecs_lib_appoint', 'is_ecs_solaire']

gen_cols = ['td001_dpe_id', 'nom_methode_dpe_norm', 'type_ventilation',
            'inertie', 'presence_climatisation', 'enr']
env_cols = [ 'td001_dpe_id', 'u_mur_ext', 'mat_mur_ext',
       'ep_mat_mur_ext', 'type_adjacence_ph', 'u_ph', 'mat_ph',
       'type_adjacence_pb', 'u_pb', 'mat_pb', 'pos_isol_mur_ext',
       'pos_isol_pb', 'pos_isol_ph', 'u_baie', 'fs_baie', 'type_vitrage_baie',
       'remplissage_baie', 'mat_baie', 'orientation_baie',
       'avancee_masque_max', 'presence_balcon']
grnv = td001[td001_cols].drop_duplicates('numero_dpe', keep='last')
grnv = grnv.merge(gen_adv[gen_cols], on='td001_dpe_id', how='left')
grnv = grnv.merge(sys_adv[sys_cols], on='td001_dpe_id', how='left')
grnv = grnv.merge(env_surf[env_surf_cols], on='td001_dpe_id', how='left')
grnv = grnv.merge(env_adv[env_cols], on='td001_dpe_id', how='left')
grnv = rename_dpe_table_light(grnv, reformat_gorenove=True)
grnv = grnv.replace('indetermine',np.nan).replace('inconnu',np.nan)

In [44]:
import generate_dpe_annexes_scripts.gorenove_scripts
reload_package(generate_dpe_annexes_scripts.gorenove_scripts)
from generate_dpe_annexes_scripts.gorenove_scripts import *

In [45]:

grnv=grnv.astype({k:v for k,v in gorenove_types.items() if k in grnv})

# generate table

In [39]:
import main_enveloppe
import generate_dpe_annexes_scripts.conversion_normalisation

import generate_dpe_annexes_scripts.td001_merge
import generate_dpe_annexes_scripts.text_matching_dict
import generate_dpe_annexes_scripts.utils
from main_enveloppe import *
reload_package(generate_dpe_annexes_scripts.trtvtables)
reload_package(generate_dpe_annexes_scripts.td008_processing)
reload_package(generate_dpe_annexes_scripts.td007_processing)
import generate_dpe_annexes_scripts.td008_processing
import generate_dpe_annexes_scripts.td007_processing
from generate_dpe_annexes_scripts.utils import reload_package

import generate_dpe_annexes_scripts.td008_processing
import generate_dpe_annexes_scripts.td007_processing

import generate_dpe_annexes_scripts.trtvtables
reload_package(generate_dpe_annexes_scripts.trtvtables)
reload_package(generate_dpe_annexes_scripts.td008_processing)
reload_package(generate_dpe_annexes_scripts.td007_processing)
reload_package(main_enveloppe)
from main_enveloppe import *


In [2]:
import main_enveloppe
import generate_dpe_annexes_scripts.conversion_normalisation

import generate_dpe_annexes_scripts.td001_merge
import generate_dpe_annexes_scripts.text_matching_dict
import generate_dpe_annexes_scripts.utils
from generate_dpe_annexes_scripts.trtvtables import DPETrTvTables
from main_enveloppe import *

In [3]:
td007 = pd.read_csv(dept_dir / 'td007_paroi_opaque.csv', dtype=str)
td003 = pd.read_csv(dept_dir / 'td003_descriptif.csv', dtype=str)
td005 = pd.read_csv(dept_dir / 'td005_fiche_technique.csv', dtype=str)
td006 = pd.read_csv(dept_dir / 'td006_batiment.csv', dtype=str)
td001 = pd.read_csv(dept_dir / 'td001_dpe.csv', dtype=str)
td001 = td001.rename(columns={'id': 'td001_dpe_id'})
td008 = pd.read_csv(dept_dir / 'td008_baie.csv', dtype=str)
td008 = td008.drop('td008_baie_id', axis=1)
td010 = pd.read_csv(dept_dir / 'td010_pont_thermique.csv', dtype=str)

# ENVELOPPE PROCESSING
surfaces_agg_essential, td008_p, td007_p, env_compo_dict, env_compo_agg_dict = run_enveloppe_processing(td001,
                                                                                                        td006,
                                                                                                        td007,
                                                                                                        td008,
                                                                                                        td010)

tv025_intermittence_type_ch not found
tv026_classe_inertie_plancher_bas not found
tv026_classe_inertie_plancher_haut not found
tv026_classe_inertie_paroi_verticale not found
tv026_classe_inertie_classe_inertie not found
tv025_intermittence_type_emetteur not found
tv029_simu_type_distribution not found


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [217]:
td001=td001.rename(columns={'id':'td001_dpe_id'})

# exploration

In [384]:
import generate_dpe_annexes_scripts.text_matching_dict
reload_package(generate_dpe_annexes_scripts.text_matching_dict)

from generate_dpe_annexes_scripts.text_matching_dict import *

# murs

In [42]:
from generate_dpe_annexes_scripts.utils import strip_accents,clean_desc_txt




td005_mur = td005.loc[td005.tr011_sous_categorie_fiche_technique_id == '9']
td005_mur.valeur_renseignee = td005_mur.valeur_renseignee.fillna('indetermine')

td005_mur.valeur_renseignee = td005_mur.valeur_renseignee.str.lower().apply(lambda x: strip_accents(x))
td005_mur.valeur_renseignee = td005_mur.valeur_renseignee.apply(lambda x:clean_desc_txt(x))




td003_mur = td003.loc[td003.tr007_type_descriptif_id == '6']
td003_mur.descriptif = td003_mur.descriptif.fillna('indetermine')

td003_mur.descriptif = td003_mur.descriptif.str.lower().apply(lambda x: strip_accents(x))
td003_mur.descriptif = td003_mur.descriptif.apply(lambda x:clean_desc_txt(x))



C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [11]:
from generate_dpe_annexes_scripts.td003_td005_text_extraction import *

ES : murs


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


ES : murs


In [6]:
env_dict_cols = {
    'td007_murs_agg': ['u_mur_exterieur_top', 'mat_mur_exterieur_top', 'ep_mat_mur_exterieur_top'],
    "td007_ph_agg": ['type_adjacence_ph_top', 'u_ph_top', 'mat_ph_top'],
    "td007_pb_agg": ['type_adjacence_pb_top', 'u_pb_top', 'mat_pb_top'],
    "td010_agg": ['pos_isol_mur_ext', 'pos_isol_pb', 'pos_isol_ph'],
    "td008_agg": ['u_baie_baie_vitree_top', 'facteur_solaire_corr_baie_vitree_top',
                  'type_vitrage_baie_vitree_top', 'remplissage_baie_vitree_top', 'mat_baie_vitree_top',
                  'orientation_baie_infer', 'avancee_masque_max',
                  "presence_balcon"],
}

In [8]:
from generate_dpe_annexes_scripts.gorenove_scripts import rename_dpe_table_light

In [13]:
from generate_dpe_annexes_scripts.advanced_enveloppe_processing import concat_mur_txt

In [14]:
td001_env = concat_mur_txt(td001_env, materiau_mur_ft=materiau_mur_ft, materiau_mur_desc=materiau_mur_desc,
                           isolation_mur_ft=isolation_mur_ft, isolation_mur_desc=isolation_mur_desc)

In [19]:
from generate_dpe_annexes_scripts.text_matching_dict import *

In [20]:
    materiau_mur_from_txt = pd.concat([materiau_mur_desc[['label', 'td001_dpe_id']],
                                           materiau_mur_ft[['label', 'td001_dpe_id']]], axis=0)

    materiau_mur_from_txt['label'] = pd.Categorical(materiau_mur_from_txt.label,
                                                    categories=list(murs_materiau_search_dict.keys()) + ['indetermine'],
                                                    ordered=True)

    materiau_mur_from_txt = materiau_mur_from_txt.sort_values(by=['td001_dpe_id', 'label'])

    materiau_mur_from_txt = materiau_mur_from_txt.drop_duplicates(subset=['td001_dpe_id'], keep='first')
    # suppression de la dénomination exact qui ne correpond qu'a une méthode de recherche plus rigide
    materiau_mur_from_txt.label = materiau_mur_from_txt.label.str.replace(' exact', '')

In [22]:
    materiau_mur_from_txt = pd.concat([materiau_mur_desc[['label', 'td001_dpe_id']],
                                           materiau_mur_ft[['label', 'td001_dpe_id']]], axis=0)


In [23]:
materiau_mur_from_txt.label.value_counts()

dalle béton                                            372253
plafond avec ou sans remplissage                          816
plafond avec ou sans remplissage sous combles             510
plafond mitoyen non deperditif                            134
combles aménagés sous rampant                              23
plafond bois sur solives bois                               4
plafond entre solives bois avec ou sans remplissage         4
plafond bois sous solives bois                              4
Name: label, dtype: int64

In [16]:
td001_env.mat_mur_ext_txt.value_counts()

Series([], Name: mat_mur_ext_txt, dtype: int64)

In [9]:
td001_env = td001[['td001_dpe_id']]

for k, v in env_dict_cols.items():
    td001_env = td001_env.merge(env_compo_agg_dict[k][v], on='td001_dpe_id', how='left')

td001_env = rename_dpe_table_light(td001_env)
final_cols = td001_env.columns

In [ ]:
print('ES : murs')
materiau_mur_ft, isolation_mur_ft = extract_td005_murs_variables(td005)
print('ES : murs')

materiau_mur_desc, isolation_mur_desc = extract_td003_murs_variables(td003)

In [28]:
grnv.baie_type_vitrage.value_counts()

double vitrage                      117176
simple vitrage                       15091
triple vitrage                        4692
INCOHERENT                            1833
survitrage                              99
brique de verre ou polycarbonate         7
Name: baie_type_vitrage, dtype: int64

In [26]:
materiau_mur_desc

,id,label,td001_dpe_id,descriptif
0,57498399,dalle béton,3893681,mur exterieur : mur en beton banche epaisseur ...
1,57500568,dalle béton,3893854,mur exterieur : mur en beton banche epaisseur ...
2,57500637,dalle béton,3893860,mur exterieur : mur en beton banche epaisseur ...
3,57500638,dalle béton,3893860,mur lnc : mur en beton banche epaisseur : 20 -...
4,57500682,dalle béton,3893864,mur lnc : mur en beton banche epaisseur : 20 -...
...,...,...,...,...
256863,187720980,plafond avec ou sans remplissage,9220358,- mur sur exterieur - u=0 233 w m².k - mur sur...
256864,187721023,plafond avec ou sans remplissage,9220362,- mur sur exterieur - u=0 233 w m².k - mur sur...
256865,187721035,plafond avec ou sans remplissage,9220363,- mur sur exterieur - u=0 233 w m².k - mur sur...
256866,189810764,plafond avec ou sans remplissage,9344232,- mur exterieur - u=0 239 w m².k - plafond sur...


In [13]:
from generate_dpe_annexes_scripts.utils_elasticsearch import search_and_affect

In [306]:

m=search_and_affect(td005_mur,id_col='id',val_col='valeur_renseignee',
                    search_dict=murs_materiau_search_dict)

materiau_mur_ft = m.merge(td005_mur[['id', 'td001_dpe_id','valeur_renseignee']], how='left')



m=search_and_affect(td005_mur,id_col='id',val_col='valeur_renseignee',
                    search_dict=isolation_search_dict)

isolation_mur_ft = m.merge(td005_mur[['id', 'td001_dpe_id']], how='left')


In [307]:

m=search_and_affect(td003_mur,id_col='id',val_col='descriptif',
                    search_dict=murs_materiau_search_dict)

materiau_mur_desc = m.merge(td003_mur[['id', 'td001_dpe_id','descriptif']], how='left')



m=search_and_affect(td003_mur,id_col='id',val_col='descriptif',
                    search_dict=isolation_search_dict)

isolation_mur_desc = m.merge(td003_mur[['id', 'td001_dpe_id']], how='left')


# plancher bas


In [5]:

td005_pb = td005.loc[td005.tr011_sous_categorie_fiche_technique_id == '10']
td005_pb.valeur_renseignee = td005_pb.valeur_renseignee.fillna('indetermine')

td005_pb.valeur_renseignee = td005_pb.valeur_renseignee.str.lower().apply(lambda x: strip_accents(x))
td005_pb.valeur_renseignee = td005_pb.valeur_renseignee.apply(lambda x:clean_desc_txt(x))




td003_pb = td003.loc[td003.tr007_type_descriptif_id == '8']
td003_pb.descriptif = td003_pb.descriptif.fillna('indetermine')

td003_pb.descriptif = td003_pb.descriptif.str.lower().apply(lambda x: strip_accents(x))
td003_pb.descriptif = td003_pb.descriptif.apply(lambda x:clean_desc_txt(x))

In [342]:

m=search_and_affect(td005_pb,id_col='id',val_col='valeur_renseignee',
                    search_dict=pb_materiau_search_dict)

materiau_pb_ft = m.merge(td005_pb[['id', 'td001_dpe_id','valeur_renseignee']], how='left')



m=search_and_affect(td005_pb,id_col='id',val_col='valeur_renseignee',
                    search_dict=isolation_search_dict)

isolation_pb_ft = m.merge(td005_pb[['id', 'td001_dpe_id']], how='left')



m=search_and_affect(td003_pb,id_col='id',val_col='descriptif',
                    search_dict=pb_materiau_search_dict)

materiau_pb_desc = m.merge(td003_pb[['id', 'td001_dpe_id','descriptif']], how='left')



m=search_and_affect(td003_pb,id_col='id',val_col='descriptif',
                    search_dict=isolation_search_dict)

isolation_pb_desc = m.merge(td003_pb[['id', 'td001_dpe_id']], how='left')


# plancher haut

In [6]:
from generate_dpe_annexes_scripts.utils import strip_accents,clean_desc_txt

td005_ph = td005.loc[td005.tr011_sous_categorie_fiche_technique_id == '11']
td005_ph.valeur_renseignee = td005_ph.valeur_renseignee.fillna('indetermine')

td005_ph.valeur_renseignee = td005_ph.valeur_renseignee.str.lower().apply(lambda x: strip_accents(x))
td005_ph.valeur_renseignee = td005_ph.valeur_renseignee.apply(lambda x:clean_desc_txt(x))

td003_ph = td003.loc[td003.tr007_type_descriptif_id == '16']
td003_ph.descriptif = td003_ph.descriptif.fillna('indetermine')

td003_ph.descriptif = td003_ph.descriptif.str.lower().apply(lambda x: strip_accents(x))
td003_ph.descriptif = td003_ph.descriptif.apply(lambda x:clean_desc_txt(x))



In [385]:

m=search_and_affect(td005_ph,id_col='id',val_col='valeur_renseignee',
                    search_dict=ph_materiau_search_dict)

materiau_ph_ft = m.merge(td005_ph[['id', 'td001_dpe_id','valeur_renseignee']], how='left')



m=search_and_affect(td005_ph,id_col='id',val_col='valeur_renseignee',
                    search_dict=isolation_search_dict)

isolation_ph_ft = m.merge(td005_ph[['id', 'td001_dpe_id']], how='left')


m=search_and_affect(td003_ph,id_col='id',val_col='descriptif',
                    search_dict=ph_materiau_search_dict)

materiau_ph_desc = m.merge(td003_ph[['id', 'td001_dpe_id','descriptif']], how='left')


m=search_and_affect(td003_ph,id_col='id',val_col='descriptif',
                    search_dict=isolation_search_dict)

isolation_ph_desc = m.merge(td003_ph[['id', 'td001_dpe_id']], how='left')


# vitrages

In [7]:

td005_fen = td005.loc[td005.tr011_sous_categorie_fiche_technique_id == '12']
td005_fen.valeur_renseignee = td005_fen.valeur_renseignee.fillna('indetermine')

td005_fen.valeur_renseignee = td005_fen.valeur_renseignee.str.lower().apply(lambda x: strip_accents(x))
td005_fen.valeur_renseignee = td005_fen.valeur_renseignee.apply(lambda x:clean_desc_txt(x))




td003_fen = td003.loc[td003.tr007_type_descriptif_id == '5']
td003_fen.descriptif = td003_fen.descriptif.fillna('indetermine')

td003_fen.descriptif = td003_fen.descriptif.str.lower().apply(lambda x: strip_accents(x))
td003_fen.descriptif = td003_fen.descriptif.apply(lambda x:clean_desc_txt(x))


In [21]:

m=search_and_affect(td005_fen,id_col='id',val_col='valeur_renseignee',
                    search_dict=type_vitrage_search_dict)

type_vitrage_ft = m.merge(td005_fen[['id', 'td001_dpe_id','valeur_renseignee']], how='left')


m=search_and_affect(td003_fen,id_col='id',val_col='descriptif',
                    search_dict=type_vitrage_search_dict)

type_vitrage_desc = m.merge(td003_fen[['id', 'td001_dpe_id','descriptif']], how='left')




In [22]:

m=search_and_affect(td005_fen,id_col='id',val_col='valeur_renseignee',
                    search_dict=type_remplissage_search_dict)

type_remplissage_ft = m.merge(td005_fen[['id', 'td001_dpe_id','valeur_renseignee']], how='left')


m=search_and_affect(td003_fen,id_col='id',val_col='descriptif',
                    search_dict=type_remplissage_search_dict)

type_remplissage_desc = m.merge(td003_fen[['id', 'td001_dpe_id','descriptif']], how='left')




In [23]:

m=search_and_affect(td005_fen,id_col='id',val_col='valeur_renseignee',
                    search_dict=materiau_baie_search_dict)

materiau_baie_ft = m.merge(td005_fen[['id', 'td001_dpe_id','valeur_renseignee']], how='left')


m=search_and_affect(td003_fen,id_col='id',val_col='descriptif',
                    search_dict=materiau_baie_search_dict)

materiau_baie_desc = m.merge(td003_fen[['id', 'td001_dpe_id','descriptif']], how='left')




In [24]:

m=search_and_affect(td005_fen,id_col='id',val_col='valeur_renseignee',
                    search_dict=orientation_baie_search_dict)

orientation_baie_ft = m.merge(td005_fen[['id', 'td001_dpe_id','valeur_renseignee']], how='left')


m=search_and_affect(td003_fen,id_col='id',val_col='descriptif',
                    search_dict=orientation_baie_search_dict)

orientation_baie_desc = m.merge(td003_fen[['id', 'td001_dpe_id','descriptif']], how='left')




# ventilation

In [435]:

td005_ventilation = td005.loc[td005.tr011_sous_categorie_fiche_technique_id == '15']
td005_ventilation.valeur_renseignee = td005_ventilation.valeur_renseignee.fillna('indetermine')

td005_ventilation.valeur_renseignee = td005_ventilation.valeur_renseignee.str.lower().apply(lambda x: strip_accents(x))
td005_ventilation.valeur_renseignee = td005_ventilation.valeur_renseignee.apply(lambda x:clean_desc_txt(x))

td003_ventilation = td003.loc[td003.tr007_type_descriptif_id == '13']
td003_ventilation.descriptif = td003_ventilation.descriptif.fillna('indetermine')

td003_ventilation.descriptif = td003_ventilation.descriptif.str.lower().apply(lambda x: strip_accents(x))
td003_ventilation.descriptif = td003_ventilation.descriptif.apply(lambda x:clean_desc_txt(x))


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [436]:

m=search_and_affect(td005_ventilation,id_col='id',val_col='valeur_renseignee',
                    search_dict=type_ventilation_search_dict)

type_ventilation_ft = m.merge(td005_ventilation[['id', 'td001_dpe_id','valeur_renseignee']], how='left')


m=search_and_affect(td003_ventilation,id_col='id',val_col='descriptif',
                    search_dict=type_ventilation_search_dict)

type_ventilation_desc = m.merge(td003_ventilation[['id', 'td001_dpe_id','descriptif']], how='left')




# climatisation

In [437]:

td005_climatisation = td005.loc[td005.tr011_sous_categorie_fiche_technique_id == '18']
td005_climatisation.valeur_renseignee = td005_climatisation.valeur_renseignee.fillna('indetermine')

td005_climatisation.valeur_renseignee = td005_climatisation.valeur_renseignee.str.lower().apply(lambda x: strip_accents(x))
td005_climatisation.valeur_renseignee = td005_climatisation.valeur_renseignee.apply(lambda x:clean_desc_txt(x))




td003_climatisation = td003.loc[td003.tr007_type_descriptif_id == '12']
td003_climatisation.descriptif = td003_climatisation.descriptif.fillna('indetermine')

td003_climatisation.descriptif = td003_climatisation.descriptif.str.lower().apply(lambda x: strip_accents(x))
td003_climatisation.descriptif = td003_climatisation.descriptif.apply(lambda x:clean_desc_txt(x))



m=search_and_affect(td005_climatisation,id_col='id',val_col='valeur_renseignee',
                    search_dict=presence_climatisation_search_dict)

presence_climatisation_ft = m.merge(td005_climatisation[['id', 'td001_dpe_id','valeur_renseignee']], how='left')


m=search_and_affect(td003_climatisation,id_col='id',val_col='descriptif',
                    search_dict=presence_climatisation_search_dict)

presence_climatisation_desc = m.merge(td003_climatisation[['id', 'td001_dpe_id','descriptif']], how='left')




# ENR

In [438]:

td005_enr = td005.loc[td005.tr011_sous_categorie_fiche_technique_id == '19']
td005_enr.valeur_renseignee = td005_enr.valeur_renseignee.fillna('indetermine')

td005_enr.valeur_renseignee = td005_enr.valeur_renseignee.str.lower().apply(lambda x: strip_accents(x))
td005_enr.valeur_renseignee = td005_enr.valeur_renseignee.apply(lambda x:clean_desc_txt(x))




td003_enr = td003.loc[td003.tr007_type_descriptif_id == '17']
td003_enr.descriptif = td003_enr.descriptif.fillna('indetermine')

td003_enr.descriptif = td003_enr.descriptif.str.lower().apply(lambda x: strip_accents(x))
td003_enr.descriptif = td003_enr.descriptif.apply(lambda x:clean_desc_txt(x))



m=search_and_affect(td005_enr,id_col='id',val_col='valeur_renseignee',
                    search_dict=enr_search_dict)

enr_ft = m.merge(td005_enr[['id', 'td001_dpe_id','valeur_renseignee']], how='left')


m=search_and_affect(td003_enr,id_col='id',val_col='descriptif',
                    search_dict=enr_search_dict)

enr_desc = m.merge(td003_enr[['id', 'td001_dpe_id','descriptif']], how='left')




# fusion

In [386]:
env_dict_cols = {
   'td007_murs_agg':['u_mur_exterieur_top','mat_mur_exterieur_top','ep_mat_mur_exterieur_top'],
        "td007_ph_agg":['type_adjacence_ph_top','u_ph_top','mat_ph_top'],
        "td007_pb_agg":['type_adjacence_pb_top','u_pb_top','mat_pb_top'],
        "td010_agg":['pos_isol_mur_ext','pos_isol_pb','pos_isol_ph'],
        "td008_agg":['u_baie_baie_vitree_top','facteur_solaire_corr_baie_vitree_top',
                                 'type_vitrage_baie_vitree_top','remplissage_baie_vitree_top','mat_baie_vitree_top','orientation_baie_infer','avancee_masque_max',
                                 "presence_balcon"],
}

In [387]:
td001_env=td001[['td001_dpe_id']]

for k,v in env_dict_cols.items():
    td001_env=td001_env.merge(env_compo_agg_dict[k][v],on='td001_dpe_id',how='left')

In [389]:
def rename_dpe_table_light(table,reformat_gorenove=False):
    rep = {'epaisseur': 'ep',
           'isolation': 'isol',
           'resistance_thermique': 'res_therm',
           'uniforme_': '',
           'facteur_solaire_corr': 'fs',
           'exterieur': 'ext',

           'chauffage': 'ch',
           'baie_baie_vitree': 'baie',
           'baie_vitree': 'baie',
           'materiaux': 'mat'}

    remove = ['_infer', '_top', '_avg', '_orientee', '_concat']
    
    compo = ['mur','pb','ph','baie','ch','ecs']
    
    cols = list()
    for col in table:
        for el in rep.items():
            col = col.replace(*el)
        for el in remove:
            col = col.replace(el, '')
        if reformat_gorenove:
            for el in compo:
                if el in col:
                    col = col.replace(el+'_','')
                    col = col.replace('_'+el,'')
                    col = el +'_'+col
            
        cols.append(col)
    table.columns = cols
    return table


In [ ]:
td001_env=td001[['td001_dpe_id']]

for k,v in env_dict_cols.items():
    td001_env=td001_env.merge(env_compo_agg_dict[k][v],on='td001_dpe_id',how='left')

td001_env=rename_dpe_table_light(td001_env)

## murs 

In [391]:
# libéllés des generateurs chauffage issues des données textes
materiau_mur_from_txt = pd.concat([materiau_mur_desc[['label', 'td001_dpe_id']],
                                 materiau_mur_ft[['label', 'td001_dpe_id']]], axis=0)

materiau_mur_from_txt['label'] = pd.Categorical(materiau_mur_from_txt.label,
                                              categories=list(murs_materiau_search_dict.keys()) + ['indetermine'],
                                              ordered=True)

materiau_mur_from_txt = materiau_mur_from_txt.sort_values(by=['td001_dpe_id', 'label'])

materiau_mur_from_txt = materiau_mur_from_txt.drop_duplicates(subset=['td001_dpe_id'], keep='first')
# suppression de la dénomination exact qui ne correpond qu'a une méthode de recherche plus rigide
materiau_mur_from_txt.label = materiau_mur_from_txt.label.str.replace(' exact', '')

td001_env=td001_env.merge(materiau_mur_from_txt.rename(columns={"label":'mat_mur_ext_txt'}),
                          on='td001_dpe_id',how='left')

# libéllés des generateurs chauffage issues des données textes
isolation_mur_from_txt = pd.concat([isolation_mur_desc[['label', 'td001_dpe_id']],
                                 isolation_mur_ft[['label', 'td001_dpe_id']]], axis=0)

isolation_mur_from_txt['label'] = pd.Categorical(isolation_mur_from_txt.label,
                                              categories=list(isolation_search_dict.keys()) + ['indetermine'],
                                              ordered=True)

isolation_mur_from_txt = isolation_mur_from_txt.sort_values(by=['td001_dpe_id', 'label'])

isolation_mur_from_txt = isolation_mur_from_txt.drop_duplicates(subset=['td001_dpe_id'], keep='first')
# suppression de la dénomination exact qui ne correpond qu'a une méthode de recherche plus rigide
isolation_mur_from_txt.label = isolation_mur_from_txt.label.str.replace(' exact', '')

td001_env=td001_env.merge(isolation_mur_from_txt.rename(columns={"label":'pos_isol_mur_ext_txt'}),
                          on='td001_dpe_id',how='left')

## plancher bas

In [392]:
# libéllés des generateurs chauffage issues des données textes
materiau_pb_from_txt = pd.concat([materiau_pb_desc[['label', 'td001_dpe_id']],
                                 materiau_pb_ft[['label', 'td001_dpe_id']]], axis=0)

materiau_pb_from_txt['label'] = pd.Categorical(materiau_pb_from_txt.label,
                                              categories=list(pb_materiau_search_dict.keys()),
                                              ordered=True)

materiau_pb_from_txt = materiau_pb_from_txt.sort_values(by=['td001_dpe_id', 'label'])

materiau_pb_from_txt = materiau_pb_from_txt.drop_duplicates(subset=['td001_dpe_id'], keep='first')
# suppression de la dénomination exact qui ne correpond qu'a une méthode de recherche plus rigide
materiau_pb_from_txt.label = materiau_pb_from_txt.label.str.replace(' exact', '')

td001_env=td001_env.merge(materiau_pb_from_txt.rename(columns={"label":'mat_pb_txt'}),
                          on='td001_dpe_id',how='left')

# libéllés des generateurs chauffage issues des données textes
isolation_pb_from_txt = pd.concat([isolation_pb_desc[['label', 'td001_dpe_id']],
                                 isolation_pb_ft[['label', 'td001_dpe_id']]], axis=0)

isolation_pb_from_txt['label'] = pd.Categorical(isolation_pb_from_txt.label,
                                              categories=list(isolation_search_dict.keys()) + ['indetermine'],
                                              ordered=True)

isolation_pb_from_txt = isolation_pb_from_txt.sort_values(by=['td001_dpe_id', 'label'])

isolation_pb_from_txt = isolation_pb_from_txt.drop_duplicates(subset=['td001_dpe_id'], keep='first')
# suppression de la dénomination exact qui ne correpond qu'a une méthode de recherche plus rigide
isolation_pb_from_txt.label = isolation_pb_from_txt.label.str.replace(' exact', '')

td001_env=td001_env.merge(isolation_pb_from_txt.rename(columns={"label":'pos_isol_pb_txt'}),
                          on='td001_dpe_id',how='left')

## plancher haut

In [393]:
# libéllés des generateurs chauffage issues des données textes
materiau_ph_from_txt = pd.concat([materiau_ph_desc[['label', 'td001_dpe_id']],
                                 materiau_ph_ft[['label', 'td001_dpe_id']]], axis=0)

materiau_ph_from_txt['label'] = pd.Categorical(materiau_ph_from_txt.label,
                                              categories=list(ph_materiau_search_dict.keys()) + ['indetermine'],
                                              ordered=True)

materiau_ph_from_txt = materiau_ph_from_txt.sort_values(by=['td001_dpe_id', 'label'])

materiau_ph_from_txt = materiau_ph_from_txt.drop_duplicates(subset=['td001_dpe_id'], keep='first')
# suppression de la dénomination exact qui ne correpond qu'a une méthode de recherche plus rigide
materiau_ph_from_txt.label = materiau_ph_from_txt.label.str.replace(' exact', '')

td001_env=td001_env.merge(materiau_ph_from_txt.rename(columns={"label":'mat_ph_txt'}),
                          on='td001_dpe_id',how='left')

# libéllés des generateurs chauffage issues des données textes
isolation_ph_from_txt = pd.concat([isolation_ph_desc[['label', 'td001_dpe_id']],
                                 isolation_ph_ft[['label', 'td001_dpe_id']]], axis=0)

isolation_ph_from_txt['label'] = pd.Categorical(isolation_ph_from_txt.label,
                                              categories=list(isolation_search_dict.keys()) + ['indetermine'],
                                              ordered=True)

isolation_ph_from_txt = isolation_ph_from_txt.sort_values(by=['td001_dpe_id', 'label'])

isolation_ph_from_txt = isolation_ph_from_txt.drop_duplicates(subset=['td001_dpe_id'], keep='first')
# suppression de la dénomination exact qui ne correpond qu'a une méthode de recherche plus rigide
isolation_ph_from_txt.label = isolation_ph_from_txt.label.str.replace(' exact', '')

td001_env=td001_env.merge(isolation_ph_from_txt.rename(columns={"label":'pos_isol_ph_txt'}),
                          on='td001_dpe_id',how='left')

## vitrage

In [394]:
# libéllés des type vitrage issues des données textes
type_vitrage_from_txt = pd.concat([type_vitrage_desc[['label', 'td001_dpe_id']],
                                 type_vitrage_ft[['label', 'td001_dpe_id']]], axis=0)

type_vitrage_from_txt['label'] = pd.Categorical(type_vitrage_from_txt.label,
                                              categories=list(type_vitrage_search_dict.keys()) + ['indetermine'],
                                              ordered=True)

type_vitrage_from_txt = type_vitrage_from_txt.sort_values(by=['td001_dpe_id', 'label'])

type_vitrage_from_txt = type_vitrage_from_txt.drop_duplicates(subset=['td001_dpe_id'], keep='first')
# suppression de la dénomination exact qui ne correpond qu'a une méthode de recherche plus rigide
type_vitrage_from_txt.label = type_vitrage_from_txt.label.str.replace(' exact', '')

td001_env=td001_env.merge(type_vitrage_from_txt.rename(columns={"label":'type_vitrage_baie_txt'}),
                          on='td001_dpe_id',how='left')

# libéllés du remplissage des vitrage issues des données textes
type_remplissage_from_txt = pd.concat([type_remplissage_desc[['label', 'td001_dpe_id']],
                                 type_remplissage_ft[['label', 'td001_dpe_id']]], axis=0)

type_remplissage_from_txt['label'] = pd.Categorical(type_remplissage_from_txt.label,
                                              categories=list(type_remplissage_search_dict.keys()) + ['indetermine'],
                                              ordered=True)

type_remplissage_from_txt = type_remplissage_from_txt.sort_values(by=['td001_dpe_id', 'label'])

type_remplissage_from_txt = type_remplissage_from_txt.drop_duplicates(subset=['td001_dpe_id'], keep='first')
# suppression de la dénomination exact qui ne correpond qu'a une méthode de recherche plus rigide
type_remplissage_from_txt.label = type_remplissage_from_txt.label.str.replace(' exact', '')

td001_env=td001_env.merge(type_remplissage_from_txt.rename(columns={"label":'remplissage_baie_txt'}),
                          on='td001_dpe_id',how='left')

# libéllés des materiau de baie issues des données textes
materiau_baie_from_txt = pd.concat([materiau_baie_desc[['label', 'td001_dpe_id']],
                                 materiau_baie_ft[['label', 'td001_dpe_id']]], axis=0)

materiau_baie_from_txt['label'] = pd.Categorical(materiau_baie_from_txt.label,
                                              categories=list(materiau_baie_search_dict.keys()) + ['indetermine'],
                                              ordered=True)

materiau_baie_from_txt = materiau_baie_from_txt.sort_values(by=['td001_dpe_id', 'label'])

materiau_baie_from_txt = materiau_baie_from_txt.drop_duplicates(subset=['td001_dpe_id'], keep='first')
# suppression de la dénomination exact qui ne correpond qu'a une méthode de recherche plus rigide
materiau_baie_from_txt.label = materiau_baie_from_txt.label.str.replace(' exact', '')

td001_env=td001_env.merge(materiau_baie_from_txt.rename(columns={"label":'mat_baie_txt'}),
                          on='td001_dpe_id',how='left')

# libéllés des orientation baie issues des données textes
orientation_baie_from_txt = pd.concat([orientation_baie_desc[['label', 'td001_dpe_id']],
                                 orientation_baie_ft[['label', 'td001_dpe_id']]], axis=0)

orientation_baie_from_txt['label'] = pd.Categorical(orientation_baie_from_txt.label,
                                              categories=list(orientation_baie_search_dict.keys()),
                                              ordered=True)

orientation_baie_from_txt = orientation_baie_from_txt.groupby('td001_dpe_id').label.apply(lambda x:' + '.join(sorted(list(set(x))))).to_frame('orientation_baie_txt')


td001_env=td001_env.merge(orientation_baie_from_txt,
                          on='td001_dpe_id',how='left')

# sub missing by text

In [395]:
td001_env_b=td001_env.copy()

In [396]:
vars_to_sub=['mat_mur_ext','pos_isol_mur','mat_ph',
 'pos_isol_ph','mat_pb',
 'pos_isol_pb','type_vitrage_baie',
 'remplissage_baie','mat_baie','orientation_baie']

for var in vars_to_sub:
    
    is_null = (td001_env[var].isnull())|(td001_env[var]=='inconnu')|(td001_env[var]=='indetermine')
    td001_env.loc[is_null,var]=td001_env.loc[is_null,var+'_txt']

# corr isolation

In [375]:
comp=pd.concat([td001_env_b.count().to_frame('before'),td001_env.count().to_frame('after')],axis=1)

In [376]:
comp['gain(%)']=((comp['after']/comp.before)*100-100).round()

In [409]:
is_non_isole = td001_env['u_mur_ext']>1.5

td001_env.loc[is_non_isole,'pos_isol_mur']='non isole'

td001_env.pos_isol_mur= td001_env.pos_isol_mur.replace('Non isolé','non isole')

is_non_isole = td001_env['u_pb']>1.5

td001_env.loc[is_non_isole,'pos_isol_pb']='non isole'

td001_env.pos_isol_pb= td001_env.pos_isol_pb.replace('Non isolé','non isole')

is_non_isole = td001_env['u_ph']>1.5

td001_env.loc[is_non_isole,'pos_isol_ph']='non isole'

td001_env.pos_isol_ph= td001_env.pos_isol_ph.replace('Non isolé','non isole')

# fusion autres

## ventilation

In [464]:
td001_gen=td001[['td001_dpe_id']].merge(td001_td006,on='td001_dpe_id',how='left')

In [465]:
# libéllés des generateurs chauffage issues des données textes
type_ventilation_from_txt = pd.concat([type_ventilation_desc[['label', 'td001_dpe_id']],
                                 type_ventilation_ft[['label', 'td001_dpe_id']]], axis=0)

type_ventilation_from_txt['label'] = pd.Categorical(type_ventilation_from_txt.label,
                                              categories=list(type_ventilation_search_dict.keys()) + ['indetermine'],
                                              ordered=True)

type_ventilation_from_txt = type_ventilation_from_txt.sort_values(by=['td001_dpe_id', 'label'])

type_ventilation_from_txt = type_ventilation_from_txt.drop_duplicates(subset=['td001_dpe_id'], keep='first')
# suppression de la dénomination exact qui ne correpond qu'a une méthode de recherche plus rigide
type_ventilation_from_txt.label = type_ventilation_from_txt.label.str.replace(' exact', '')

td001_gen=td001_gen.merge(type_ventilation_from_txt.rename(columns={"label":'type_ventilation_txt'}),
                          on='td001_dpe_id',how='left')

## climatisation

# libéllés des generateurs chauffage issues des données textes
presence_climatisation_from_txt = pd.concat([presence_climatisation_desc[['label', 'td001_dpe_id']],
                                 presence_climatisation_ft[['label', 'td001_dpe_id']]], axis=0)

presence_climatisation_from_txt['label'] = pd.Categorical(presence_climatisation_from_txt.label,
                                              categories=list(presence_climatisation_search_dict.keys()) + ['indetermine'],
                                              ordered=True)

presence_climatisation_from_txt = presence_climatisation_from_txt.sort_values(by=['td001_dpe_id', 'label'])

presence_climatisation_from_txt = presence_climatisation_from_txt.drop_duplicates(subset=['td001_dpe_id'], keep='first')
# suppression de la dénomination exact qui ne correpond qu'a une méthode de recherche plus rigide
presence_climatisation_from_txt.label = presence_climatisation_from_txt.label.str.replace(' exact', '')

td001_gen=td001_gen.merge(presence_climatisation_from_txt.rename(columns={"label":'presence_climatisation'}),
                          on='td001_dpe_id',how='left')

## ENR

# libéllés des generateurs chauffage issues des données textes
enr_from_txt = pd.concat([enr_desc[['label', 'td001_dpe_id']],
                                 enr_ft[['label', 'td001_dpe_id']]], axis=0)
enr_from_txt.label =enr_from_txt.label.replace('solaire thermique','solaire thermique (ecs)')
enr_from_txt['label'] = pd.Categorical(enr_from_txt.label,
                                              categories=list(enr_search_dict.keys()),
                                              ordered=True)

enr_from_txt = enr_from_txt.groupby('td001_dpe_id').label.apply(lambda x:' + '.join(sorted(list(set(x))))).to_frame('enr')


td001_gen=td001_gen.merge(enr_from_txt,
                          on='td001_dpe_id',how='left')

is_null=td001_gen['type_ventilation'].isnull()
td001_gen.loc[is_null,'type_ventilation']=td001_gen.loc[is_null,'type_ventilation_txt']

td001_gen.presence_climatisation=(~td001_gen.presence_climatisation.isnull()).astype(int)

    td001['nom_methode_dpe_norm'] = 'NON DEFINI'

    nom_dpe = td001.nom_methode_dpe.copy().str.lower()

    is_3cl = nom_dpe.str.contains('cl')
    is_facture = nom_dpe.str.contains('facture')
    is_thc = nom_dpe.str.startswith('th')
    is_vierge = nom_dpe.str.contains('vierge')
    s_version = td001.version_methode_dpe.str.lower().fillna('')
    v2012 = s_version.str.contains('2012|1.3')

    td001.loc[is_3cl & v2012, 'nom_methode_dpe_norm'] = '3CL 2012'
    td001.loc[is_3cl & (~v2012), 'nom_methode_dpe_norm'] = '3CL 2005'
    td001.loc[is_facture, 'nom_methode_dpe_norm'] = 'FACTURE'
    td001.loc[is_thc, 'nom_methode_dpe_norm'] = 'THBCE(RT2012)/THC(RT2005)'
    td001.loc[is_vierge, 'nom_methode_dpe_norm'] = 'DPE vierge'
    periode_construction = pd.cut(td001.annee_construction.astype(float),
                                  [-100000, 1700, 1948, 1970, 1988, 1999, 2005, 2012, 2020, 2100],
                                  labels=['bad inf', '<1948', '1949-1970', '1970-1988', '1989-1999', '2000-2005',
                                          '2006-2012', '>2012', 'bad sup'])

In [477]:
td001['periode_construction']=periode_construction

In [478]:
td001['periode_construction'].value_counts()

<1948        48751
>2012        29495
1970-1988    26391
bad inf      11383
1949-1970     9452
2006-2012     7199
2000-2005     6792
1989-1999     6509
bad sup          2
Name: periode_construction, dtype: int64

In [474]:
td001[['periode_construction','annee_construction']]

,periode_construction,annee_construction
0,2006-2012,2006
1,<1948,1948
2,2006-2012,2007
3,2006-2012,2006
4,1970-1988,1982
...,...,...
145974,1989-1999,1994
145975,1970-1988,1978
145976,<1948,1947
145977,1970-1988,1974


# debug

In [ ]:
grnv.count()

In [42]:
grnv.loc[grnv.nom_methode_dpe=='FACTURE'].count().sort_values()

avancee_masque_max           32
mur_mat_ext                  37
mur_ep_mat_ext               52
perc_surf_vitree_ext         94
mur_u_ext                   114
baie_fs                     201
baie_u                      201
presence_balcon             218
pb_type_adjacence           225
pb_u                        225
ph_type_adjacence           226
ph_u                        226
enr                         654
inertie                    1772
ecs_gen_lib_appoint        2076
ch_gen_lib_appoint         6184
baie_orientation          34420
type_ventilation          36912
baie_remplissage          43709
ch_type_inst              45036
ecs_type_inst             45395
pb_pos_isol               46597
ph_pos_isol               49046
ph_mat                    64828
pb_mat                    66465
mur_pos_isol_ext          68176
baie_type_vitrage         70976
ecs_type_ener             72280
baie_mat                  73812
ch_type_ener              74415
ch_gen_lib                74917
ch_gen_l

In [13]:
grnv.loc[grnv.nom_methode_dpe=='FACTURE'].mur_mat_ext.fillna('NA').value_counts()

NA                                                                   76307
murs en béton banché                                                    22
murs en blocs de béton creux                                             7
murs en briques pleines simples                                          3
murs en pierre de taille et moellons avec remplissage tout venant        3
murs en briques creuses                                                  1
béton cellulaire                                                         1
Name: mur_mat_ext, dtype: int64

In [16]:
env.loc[env.td001_dpe_id=='1737']

,Unnamed: 0,td001_dpe_id,u_mur_ext,mat_mur_ext,ep_mat_mur_ext,type_adjacence_ph,u_ph,mat_ph,type_adjacence_pb,u_pb,...,pos_isol_pb,pos_isol_ph,u_baie,fs_baie,type_vitrage_baie,remplissage_baie,mat_baie,orientation_baie,avancee_masque_max,presence_balcon
5,5,1737,NaN,NaN,NaN,NaN,NaN,dalle béton,NaN,NaN,...,non isole,ITI,NaN,NaN,triple vitrage,air sec,bois,nord + sud,NaN,NaN


In [12]:
grnv.loc[grnv.nom_methode_dpe=='FACTURE'][[ 'td001_dpe_id', 'tr002_type_batiment_id']+[col for col in grnv if 'mur' in col]]

,td001_dpe_id,tr002_type_batiment_id,mur_u_ext,mur_mat_ext,mur_ep_mat_ext,mur_pos_isol_ext
5,1737,1,NaN,NaN,NaN,ITI
6,1989,2,NaN,NaN,NaN,ITI
7,1991,2,NaN,NaN,NaN,ITI
9,2124,2,NaN,NaN,NaN,non isole
10,2148,1,NaN,NaN,NaN,non isole
...,...,...,...,...,...,...
145973,9429809,2,NaN,NaN,NaN,ITI
145974,9429928,2,NaN,NaN,NaN,isole
145976,9430797,2,NaN,NaN,NaN,ITI
145977,9431074,2,NaN,NaN,NaN,ITI


In [324]:
dpe_list = td001_env.loc[td001_env.presence_climatisation_txt=='presence_climatisation'].td001_dpe_id.tolist()

In [325]:

td005_climatisation.loc[td005_climatisation.td001_dpe_id.isin(dpe_list)][['td001_dpe_id','valeur_renseignee']].to_dict(orient='records')

[{'td001_dpe_id': '7549',
  'valeur_renseignee': '- refroidissement 1 : energie electrique surface climatisee : 450m²)'},
 {'td001_dpe_id': '8560',
  'valeur_renseignee': 'refroidissement r clim : 3 energie : electrique'},
 {'td001_dpe_id': '28803',
  'valeur_renseignee': 'refroidissement r clim : 5 energie : electrique'},
 {'td001_dpe_id': '55490',
  'valeur_renseignee': 'refroidissement r clim : 3 energie : electrique'},
 {'td001_dpe_id': '56738',
  'valeur_renseignee': 'refroidissement r clim : 3 energie : electrique'},
 {'td001_dpe_id': '73519',
  'valeur_renseignee': 'refroidissement r clim : 3 energie : electrique'},
 {'td001_dpe_id': '80437',
  'valeur_renseignee': 'individuelle electrique - surface climatisee : 45 m²'},
 {'td001_dpe_id': '90527',
  'valeur_renseignee': '- refroidissement 1 : energie electrique surface climatisee : 110m²)'},
 {'td001_dpe_id': '103281',
  'valeur_renseignee': '- refroidissement 1 : energie electrique surface climatisee : 65m²)'},
 {'td001_dpe_id'

In [294]:
td003_climatisation.loc[td003_climatisation.td001_dpe_id.isin(dpe_list)][['td001_dpe_id','descriptif']].to_dict(orient='records')

[{'td001_dpe_id': '9557', 'descriptif': '- sans objet'},
 {'td001_dpe_id': '11152', 'descriptif': '- sans objet'},
 {'td001_dpe_id': '11161', 'descriptif': '- sans objet'},
 {'td001_dpe_id': '12665', 'descriptif': '- sans objet'},
 {'td001_dpe_id': '14950', 'descriptif': '- sans objet'},
 {'td001_dpe_id': '19430', 'descriptif': '- sans objet'},
 {'td001_dpe_id': '19577', 'descriptif': '- sans objet'},
 {'td001_dpe_id': '20752', 'descriptif': '- sans objet'},
 {'td001_dpe_id': '20859', 'descriptif': '- sans objet'},
 {'td001_dpe_id': '21917', 'descriptif': '- sans objet'},
 {'td001_dpe_id': '22036', 'descriptif': '- sans objet'},
 {'td001_dpe_id': '22134', 'descriptif': '- sans objet'},
 {'td001_dpe_id': '22242', 'descriptif': '- sans objet'},
 {'td001_dpe_id': '22278', 'descriptif': '- sans objet'},
 {'td001_dpe_id': '22747', 'descriptif': '- sans objet'},
 {'td001_dpe_id': '30242', 'descriptif': '- sans objet'},
 {'td001_dpe_id': '33548', 'descriptif': '- sans objet'},
 {'td001_dpe_id

In [175]:
td003_fen.loc[td003_fen.td001_dpe_id=="304"].to_dict()

{'id': {19: '3333', 20: '3334', 21: '3335'},
 'td001_dpe_id': {19: '304', 20: '304', 21: '304'},
 'tr007_type_descriptif_id': {19: '5', 20: '5', 21: '5'},
 'descriptif': {19: 'porte-fenetres coulissantes nord simple vitrage avec volets roulant pvc tablier 12mm)',
  20: 'fenetres battantes sud simple vitrage avec jalousie accordeon',
  21: 'fenetres battantes sud simple vitrage avec jalousie accordeon'},
 'est_efface': {19: '0', 20: '0', 21: '0'}}

In [174]:
simple = td005_fen.valeur_renseignee.str.contains('simple')

In [175]:
td005_fen.loc[td005_fen.td001_dpe_id.isin(dpe_list)&simple][['td001_dpe_id','valeur_renseignee']].to_dict(orient='records')

[{'td001_dpe_id': '1989',
  'valeur_renseignee': '- fenetre cuisine : fenetre battante ; en bois 5 875 m²) avec simple vitrage donnant sur l exterieur b = 1) orientation est verticale x ?75 )) ; dormant de 5 cm au nu interieur avec absence de fermeture ; absence de joints ; baie situee en fond de balcon ou fond ou flanc de loggia avec avancee entre 1 et 2 metres ; uw = 4 7 w m².k ujn = 0 w m².k ubaie = 4 7 w m².k - fenetres sejour et chambre : 3 fenetre battante ; en pvc3 x 4 725 m²) avec double vitrage remplissage air sec - 12 mm) donnant sur l exterieur b = 1) orientation est verticale x ?75 )) ; dormant de 10 cm au nu interieur avec absence de fermeture ; presence de joints ; uw = 2 7 w m².k ujn = 0 w m².k ubaie = 2 7 w m².k'},
 {'td001_dpe_id': '1991',
  'valeur_renseignee': '- fenetre cuisine : fenetre battante ; en bois 5 875 m²) avec simple vitrage donnant sur l exterieur b = 1) orientation est verticale x ?75 )) ; dormant de 5 cm au nu interieur avec absence de fermeture ; abse

In [132]:
td005_ph.valeur_renseignee.to_dict()

{12: 'dalle beton non isole donnant sur un local chauffe surface : 39 m² donnant sur : un local chauffe u : 2 w m² c b : 0',
 33: 'plafond entre solives metalliques avec ou sans remplissage non isole donnant sur un local chauffe surface : 64.34 m² donnant sur : un local chauffe u : 1.4 w m² c b : 0',
 51: 'plafond total : - type de plancher haut : dalle beton - surface m²) : 82 - u w m²k) : 0 3 - donne sur : local chauffe - coefficient de reduction des deperditions : 0 - description de l isolation : - isolation inconuue',
 71: 'dalle beton non isole donnant sur un local chauffe surface : 39 m² donnant sur : un local chauffe u : 2 w m² c b : 0',
 91: '- plafond n 1 dalle de beton non isole',
 108: 'plafond bois non isole donnant sur l exterieur surface : m² donnant sur : l exterieur u : 2 w m² c b : 1',
 110: 'plafond bois non isole donnant sur des combles perdus surface : m² donnant sur : un comble fortement ventile u : 2 w m² c b : 0 6',
 120: 'dalle beton donnant sur l exterieur avec

In [59]:
(8592*(11*40**(2/3)+0.015*24000))/(1000*70)

59.979129398512825

In [57]:
id_="9429626"

In [58]:
env.loc[env.mat_mur_ext.isnull()].td001_dpe_id

18          1799
24          2389
26          2685
29          2792
30          2852
          ...   
97947    9429354
97948    9429434
97949    9429562
97951    9429626
97953    9429951
Name: td001_dpe_id, Length: 18852, dtype: object

In [59]:
td005_mur.loc[td005_mur.td001_dpe_id==id_].valeur_renseignee.to_dict()

{}

In [60]:
td005_fen.loc[td005_fen.td001_dpe_id==id_].valeur_renseignee.to_dict()

NameError: name 'td005_fen' is not defined

In [19]:
td003_fen.loc[td003_fen.td001_dpe_id==id_].descriptif.to_dict()

{65: 'fenetres double vitrage menuiserie pvc avec lame d air 8 mm',
 67: 'fenetres de toit double vitrage menuiserie bois avec lame d air 15 mm',
 68: 'fenetres simple vitrage menuiserie bois',
 69: 'portes-fenetres coulissantes triple vitrage avec lame d air 10 mm'}

In [20]:
td003_pb.loc[td003_pb.td001_dpe_id==id_].descriptif.to_dict()

{61: 'plancher non isole donnant sur un sous sol',
 62: 'plancher donnant sur un terre-plein'}

In [21]:
td003_mur.loc[td003_mur.td001_dpe_id==id_].descriptif.to_dict()

{57: 'pierre non isolee donnant sur l exterieur',
 66: 'beton donnant sur l exterieur avec isolation interieure'}

In [22]:
is_=td005_mur.valeur_renseignee.str.contains(' bois')
td005_mur.loc[is_,'valeur_renseignee'].to_dict()

{2525: 'mur bois avec remplissage tout venant non isole donnant sur l exterieur surface : 0 m² donnant sur : l exterieur u : 1 7 w m² c b : 1',
 7108: 'mur bois avec remplissage tout venant donnant sur l exterieur avec isolation interieure realisee entre 2001 et 2005) surface : 72 m² donnant sur : l exterieur u : 0 4 w m² c b : 1',
 9842: 'pans de bois donnant sur l exterieur avec isolation interieure 10 cm) surface : 0 m² donnant sur : l exterieur u : 0.4 w m² c b : 1',
 12596: '- mur n 1 mur en pierre de taille moellons ep 50cm non isole - mur n 2 mur en pans de bois avec remplissage ep <=8cm non isole',
 13975: 'pans de bois non isole donnant sur des circulations communes sans ouverture directe sur l exterieur surface : 10 m² donnant sur : des circulations communes sans ouverture directe sur l exterieur u : 2 w m² c b : 0.8',
 14823: 'pans de bois non isole donnant sur l exterieur surface : 60 m² donnant sur : l exterieur u : 2 w m² c b : 1',
 14873: 'pans de bois non isole donnant 

# debug search

In [328]:
def generate_instruction_from_list(char_list):
    
    new_char_list=list()
    for char in char_list:
        if isinstance(char,str):
            new_char_list.append(char)
        elif isinstance(char,tuple):
            new_char_list.append('('+') OR ('.join(char)+')')
            
    search_instruction = '('+') AND ('.join(new_char_list) +')'
    search_instruction = search_instruction.replace('AND (NOT) AND','AND NOT')
    search_instruction = search_instruction.replace('(NOT) AND','NOT')

    return search_instruction

In [335]:
toto="""
systeme de production d ecs principal surface couverte = 60.0 m2 ancien ballon electrique installe en 2007. les pieces desservies sont contigues. la production est en volume chauffe. production : indiv.  ecs solaire : non  vs = 1 x 200 
"""
toto="ITI+ITR"
toto="4/16/4"
toto="double vitrage16mm"
toto='porte-fenetres coulissantes nord simple vitrage avec volets roulant pvc tablier 12mm) surface : 4.63 m² orientation : nord inclinaison : 75 ujn : 3.5 w m² c uw : 4.8 w m² c b : 1'

toto='fenetres battantes sud simple vitrage avec jalousie accordeon surface : 1.82 m² orientation : sud inclinaison : 75 ujn : 4 w m² c uw : 4.7 w m² c b : 1'
toto='dalle beton non isole donnant sur un local chauffe surface : 39 m² donnant sur : un local chauffe u : 2 w m² c b : 0'

In [336]:
index_name = 'dpe_extract_text'

In [337]:
search_instruction =generate_instruction_from_list(isolation_search_dict['isole exact'])

In [208]:
search_instruction='(("double vitrage") OR (dv) OR (\"4/16/4"))'

In [272]:
search_instruction = 'ventilo'

In [300]:
from generate_dpe_annexes_scripts.utils_elasticsearch import *

In [301]:
import time

In [338]:
search_body = {
    "query": {
        "query_string": {
            "query": search_instruction,
            "default_field": "text_to_analyze"
        },

    },

}

es_client = setup_es_client(index_name)
bulk(es_client, gendata(index_name,['id'],[toto]))

time.sleep(1)
a_dict=es_client.search(index=index_name, body=search_body,size=500000)
a_dict

{'took': 0,
 'timed_out': False,
 '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [213]:
a_dict=es_client.search(index=index_name, body=search_body,size=500000)
a_dict

{'took': 0,
 'timed_out': False,
 '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.8630463,
  'hits': [{'_index': 'dpe_extract_text',
    '_type': '_doc',
    '_id': 'QSDWQncBb5ozNkGOjliE',
    '_score': 0.8630463,
    '_source': {'id': 'id',
     'text_to_analyze': 'fenetres fixes bois orientee nord simple vitrage surface : 0 41 m² orientation : nord inclinaison : 75 ujn : 4 7 w m² c uw : 4 7 w m² c b : 1 4/12/4'}}]}}

In [253]:
bulk(es_client, gendata(index_name,['id'],[toto]))


(1, [])

In [53]:
es_client

<Elasticsearch([{}])>

In [54]:
a_dict

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.8630463,
  'hits': [{'_index': 'dpe_extract_text',
    '_type': '_doc',
    '_id': 'MeKchHYBLPwkNvcNz20A',
    '_score': 0.8630463,
    '_source': {'id': 'id',
     'text_to_analyze': '\nsysteme de production d ecs principal surface couverte = 60.0 m2 ancien ballon electrique installe en 2007. les pieces desservies sont contigues. la production est en volume chauffe. production : indiv.  ecs solaire : non  vs = 1 x 200 \n'}}]}}

In [ ]:

s_all = search_from_search_dict(es_client,search_dict,index_name=index_name)

In [46]:
m_ch = m

In [81]:
id_=m_ch.vr_source_ft.str.contains('air')&m_ch.vr_source_ft.str.contains('eau')&m_ch.vr_source_ft.str.contains('pac')

In [82]:
m_ch.loc[id_].to_dict(orient='records')

[{'td001_dpe_id': '1009801',
  'gen_ch_lib_ft': 'pac air/air',
  'vr_source_ft': 'installation de chauffage  283 8\xa0m²)\xa0:\r - chauffage 1 : pac air air split multisplit  energie : electrique)  avec equipement d intermittence central sans minimum de temperature  absence de regulation par piece  reseau de distribution isole',
  'type_installation_ch_ft': 'indetermine',
  'energie_ch_ft': 'Electricité non renouvelable',
  'gen_ch_lib_desc': 'pac air/air',
  'vr_source_desc': 'installation de chauffage \r - chauffage 1  pac air air split multisplit  energie electrique  reseau individuel',
  'type_installation_ch_desc': 'Chauffage Individuel',
  'energie_ch_desc': 'Chauffage Individuel',
  'gen_ch_lib_infer_concat': nan,
  'mix_energetique_ch': nan,
  'type_installation_ch_concat': nan,
  'nom_methode_dpe_norm': 'FACTURE'},
 {'td001_dpe_id': '1009804',
  'gen_ch_lib_ft': 'pac air/air',
  'vr_source_ft': 'installation de chauffage  283 8\xa0m²)\xa0:\r - chauffage 1 : pac air air split m

In [76]:
m_ch.loc[m_ch.gen_ch_lib_ft.fillna('na').str.contains('air/eau')]

,td001_dpe_id,gen_ch_lib_ft,vr_source_ft,type_installation_ch_ft,energie_ch_ft,gen_ch_lib_desc,vr_source_desc,type_installation_ch_desc,energie_ch_desc,gen_ch_lib_infer_concat,mix_energetique_ch,type_installation_ch_concat,nom_methode_dpe_norm


In [77]:
m_ch.gen_ch_lib_<Q<

,td001_dpe_id,gen_ch_lib_ft,vr_source_ft,type_installation_ch_ft,energie_ch_ft,gen_ch_lib_desc,vr_source_desc,type_installation_ch_desc,energie_ch_desc,gen_ch_lib_infer_concat,mix_energetique_ch,type_installation_ch_concat,nom_methode_dpe_norm
0,1000015,convecteurs electriques nfc,convecteur electrique nfc :\r - type de produ...,Chauffage Individuel,Electricité non renouvelable,convecteurs electriques nfc,convecteur electrique nfc\r,indetermine,indetermine,convecteurs electriques nfc,Electricité non renouvelable,Chauffage Individuel,3CL 2012
1,1000040,convecteurs electriques nfc,convecteurs electriques nfc systeme individue...,Chauffage Individuel,Electricité non renouvelable,convecteurs electriques nfc,convecteurs electriques nfc systeme individuel),Chauffage Individuel,Chauffage Individuel,autres emetteurs a effet joule,Electricité non renouvelable,Chauffage Individuel,3CL 2012
2,1000043,chaudiere fioul indetermine,chaudiere individuelle fioul installee apres 1...,Chauffage Individuel,Fioul domestique,chaudiere fioul indetermine,chaudiere individuelle fioul installee apres 1991,Chauffage Individuel,Chauffage Individuel,NaN,NaN,NaN,FACTURE
3,1000076,panneaux rayonnants electriques nfc,panneaux rayonnants nfc avec programmateur<br ...,indetermine,indetermine,panneaux rayonnants electriques nfc,panneaux rayonnants nfc avec programmateur,indetermine,indetermine,autres emetteurs a effet joule,Electricité non renouvelable,Chauffage Individuel,3CL 2012
4,1000112,chaudiere fioul indetermine,chaudiere individuelle fioul installee entre 1...,Chauffage Individuel,Fioul domestique,chaudiere fioul indetermine,chaudiere individuelle fioul installee entre 1...,Chauffage Individuel,Chauffage Individuel,NaN,NaN,NaN,FACTURE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145974,9402027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FACTURE
145975,9410001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FACTURE
145976,9421242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3CL 2012
145977,9421904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FACTURE


In [61]:
m_ch.gen_ch_lib_ft.value_counts().to_dict()

{'reseau de chaleur': 20592,
 'chaudiere gaz indetermine': 18914,
 'indetermine': 11751,
 'chaudiere gaz basse temperature': 10978,
 'chaudiere gaz standard': 10088,
 'convecteurs electriques nfc': 9708,
 'chaudiere gaz condensation': 8251,
 'autres emetteurs a effet joule': 8158,
 'radiateurs electriques': 6936,
 'panneaux rayonnants electriques nfc': 4599,
 'chaudiere fioul indetermine': 3706,
 'chaudiere energie indetermine indetermine': 2772,
 'chaudiere fioul standard': 1348,
 'pac air/air': 1155,
 'chaudiere energie indetermine basse temperature': 647,
 'chaudiere energie indetermine condensation': 454,
 'chaudiere fioul basse temperature': 441,
 'pac eau/eau': 397,
 'plafonds/planchers rayonnants electriques nfc': 386,
 'convecteurs electriques nfc + panneaux rayonnants electriques nfc': 246,
 'poele ou insert bois': 230,
 'convecteurs electriques nfc + radiateurs electriques': 175,
 'chaudiere bois': 120,
 'pac geothermique': 116,
 'panneaux rayonnants electriques nfc + radiate

In [53]:
s=m_ch.loc[m_ch.nom_methode_dpe_norm=='FACTURE'].gen_ch_lib_ft.fillna('indetermine').value_counts()

In [57]:
s.loc['indetermine']/s.sum()

0.15338467987006182

In [55]:
(s=='indetermine').mean()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


0.0

In [35]:
m_ch.groupby('nom_methode_dpe_norm').gen_ch_lib_ft.value_counts()

nom_methode_dpe_norm  gen_ch_lib_ft                                                              
3CL 2005              convecteurs electriques nfc                                                    45
                      radiateurs electriques                                                         15
                      panneaux rayonnants electriques nfc                                            13
                      chaudiere gaz basse temperature                                                12
                      chaudiere gaz standard                                                          9
                                                                                                     ..
FACTURE               chaudiere energie indetermine indetermine + convecteurs electriques nfc         1
                      chaudiere gaz basse temperature + chaudiere gaz indetermine                     1
                      chaudiere gaz indetermine + indetermine         

In [ ]:
m.type_installation_ch_ft.value_counts()

In [ ]:
m.loc[m.gen_ch_lib_ft=='indetermine'].head(470).to_dict(orient='records')